In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

/data0/matthew/Software/NCSA/HAL_Projects/tf_estimator_tests/estimator_test_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 1:
    tf.config.set_visible_devices(gpus[1], 'GPU')

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True)

2022-08-10 13:23:50.424544: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7369 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1


In [4]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

In [5]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [9]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 3s 2ms/step - loss: 0.3566 - sparse_categorical_accuracy: 0.9016 - val_loss: 0.1943 - val_sparse_categorical_accuracy: 0.9431
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1629 - sparse_categorical_accuracy: 0.9540 - val_loss: 0.1343 - val_sparse_categorical_accuracy: 0.9598
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1164 - sparse_categorical_accuracy: 0.9664 - val_loss: 0.1044 - val_sparse_categorical_accuracy: 0.9677
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0900 - sparse_categorical_accuracy: 0.9743 - val_loss: 0.0966 - val_sparse_categorical_accuracy: 0.9703
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0728 - sparse_categorical_accuracy: 0.9792 - val_loss: 0.0849 - val_sparse_categorical_accuracy: 0.9730
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0603 - sparse_categorical_accu

In [10]:
del model

In [11]:
del ds_test

In [12]:
del ds_train
del ds_info